# INTERMEDIATE INTERACTIVE DATA VISUALIZATION

Now that we have a basic understanding of the Holoviews & Panel libraries thanks to the Basic Interactive Data Visualization, let's build a more complex User Interface to explore the output data from ACCESS models.

If you have any questions, suggestions or comments, feel free to send me an email to Alfonso Acosta Gonçalves (a.acostagoncalves@unsw.edu.au).

# Imports & Data Loading

The code must be run on **Kernel 3-19.10**, which is the only one where panel & holoviews have been installed.

In [ ]:
%matplotlib inline

import calendar
import cartopy
import cartopy.crs as ccrs
import cmocean.cm as cmo
import holoviews as hv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import panel as pn
import xarray as xr

# Disable warning messages
import warnings
warnings.filterwarnings('ignore')

from bisect import bisect_left
from bokeh.palettes import brewer
from matplotlib import colors
from palettable.cartocolors.qualitative import Prism_8

# Enable panels & the bokeh back-end for Holoviews
pn.extension()
hv.extension('bokeh')

Functions' definitions, explained in the code as they're being used.

In [ ]:
def closest_lat(sel_lat):
    """
    Returns closest latitude to sel_lat in yt coordinates (assumes dsm.yt_ocean is sorted)
    If two numbers are equally close, return the smallest.
    
    Parameters:
    -----------
        sel_lat: float with the selected latitude
    """
    pos = bisect_left(dsm.yt_ocean, sel_lat)
    if pos == 0:
        return dsm.yt_ocean[0].values
    if pos == len(dsm.yt_ocean):
        return dsm.yt_ocean[-1].values
    before = dsm.yt_ocean[pos - 1].values
    after = dsm.yt_ocean[pos].values
    if after - sel_lat < sel_lat - before:
       return after
    else:
       return before

In [ ]:
def closest_lon(sel_lon):
    """
    Returns closest longitude to sel_lon in xt coordinates (assumes dsm.xt_ocean is sorted)
    If two numbers are equally close, return the smallest.
    
    Parameters:
    -----------
        sel_lon: float with the selected longitude
    """
    pos = bisect_left(dsm.xt_ocean, sel_lon)
    if pos == 0:
        return dsm.xt_ocean[0].values
    if pos == len(dsm.xt_ocean):
        return dsm.xt_ocean[-1].values
    before = dsm.xt_ocean[pos - 1].values
    after = dsm.xt_ocean[pos].values
    if after - sel_lon < sel_lon - before:
       return after
    else:
       return before

In [ ]:
def set_inactive_tool(plot, element): 
    """
    For Bokeh plots, deactivates the Hover tool on initial rendering
    """
    plot.state.toolbar.active_inspect = None 

Data loading:

In [ ]:
# Open files
dsm = xr.open_dataset('/home/561/ag9076/payu/1deg_maru/archive/output030/ocean/ocean_month.nc')
dso = xr.open_dataset('/home/561/ag9076/payu/1deg_maru/archive/output030/ocean/ocean.nc')

dso_c = xr.open_dataset('/g/data3/hh5/tmp/cosima/access-om2/1deg_jra55_ryf9091_spinup1_B1/output051/ocean/ocean.nc')

# 1. BUILDING ON THE BASIC VISUALIZATION

Let's say we want to investigate what's happening to ocean temperatures around Antarctica. Taking a look at the data, we can see that temperature is a 4-dimensional data:

In [ ]:
dso.temp

Since we want to look at the Southern Ocean, we're just interested in latitudes below 45°S. And, since we only have yearly data for 5 years, let's do an average of the temperature during those 5 years to smooth out the variability.

In [ ]:
# S.O. Temperature averaged through time
ds_temp = dso.temp.sel(yt_ocean=slice(-78,-45)).mean('time')
ds_temp -= 273.15 # To put temperatures into the more useful °C
ds_temp

Now this time, instead of letting Holoviews do all the work for us, we're going to take control and say which widgets we want, and how do we want the plots to react to them. Let's start by "rebuilding" one of the basic interactive plots we already know, so that we can understand better what is happening.

Since we got rid of the time dimension, we can use depth to provide some interactivity with the plot, and explore temperature changes with depth.

In [ ]:
hv.extension('bokeh') 

cmap_temp = "cmo.thermal" # Plot's colormap 

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS

# Widget to select the DEPTH. We must manually create a 'panel.widget' to select the depth. 
# Since the available depth values are not evenly distributed, we'll need to create a "discrete slider",
# in which we can specify a list with the values that can be selected via the 'options' argument.
# 'value' specifies which is the selected value by default, when the plot is loaded.
# And since the depth values are extremely long, we can format them to 2 decimal places with a standard
# format string.
# (see https://panel.pyviz.org/user_guide/Widgets.html for all widgets available)
depth_slider_temp = pn.widgets.DiscreteSlider(name='Depth', 
                                              options=ds_temp.st_ocean.values.tolist(), 
                                              value=ds_temp.st_ocean.values[0], 
                                              width=500, 
                                              formatter='%.2f (m)')

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS' LINKS

# With the widget already created, we now declare a function to update the plot based on the value
# selected on the widget. The function must be called every time we move the slider, to create 
# a new plot based on the selected depth.
# To tell the kernel to call the function whenever the widget value changes, we use a decorator:
# '@pn.depends' tells the kernel to watch for changes on depth_slider_temp, and pass its value to the
# argument 'depth'.

# !!! @pn.depends MUST BE DIRECTLY ABOVE THE FUNCTION DEFINITION, no spaces in between, 
# or the decorator will not work.
@pn.depends(depth=depth_slider_temp)
def update_plot_temp(depth):
    # We create a QuadMesh as before, giving it the data to plot, and which are the X & Y dimensions in the plot
    plot = hv.QuadMesh(ds_temp.loc[depth], 
                       kdims=["xt_ocean", "yt_ocean"])
    plot.opts(cmap=cmap_temp,
              title='Temperature at %.2f(m) depth'% (depth), ylabel='Latitude (°N)', xlabel='Longitude (°E)',
              colorbar=True, clabel="°C", 
              width=900, height=250, bgcolor='Silver', 
              framewise=True, # Tell Holoviews to adapt the frame (the axes) to each specific plot
              tools=['hover'], toolbar="above") # Add bokeh's hover tool, and place the toolbar above the plot
    return plot

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# PLOT

# TEMPERATURE ANTARCTIC PLOT
plot_temp = hv.DynamicMap(update_plot_temp) # Create a Holoviews's dynamic map from the QuadMesh generated.
                                            # The dynamic map will refresh automatically whenever
                                            # the update_plot_temp function is called by the kernel.

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# Build the layout via panel, using Spacers as padding amongst elements. The rest is pretty self-explanatory,
# pn.Column creates a column with all the elements passed, while pn.Row builds rows
pn.Column(pn.Row(pn.Spacer(width=50), depth_slider_temp),
          pn.Spacer(height=5), # Provide some "air" between the widget and the plot
          plot_temp)

Voilà! Our familiar interactive temperature plot is ready for use once again ^_^

# 2. BUILDING A CROSS-SECTION

Now that we have successfully rebuilt what Holoviews already provided us in much fewer lines, we can use this knowledge to build something more interesting to explore our data.

Typically in oceanography, we are interested in looking at cross-sections in the ocean. Wouldn't it be nice to simply select on our plot where do we want to see the cross-section, instead of guessing longitudes, changing code & re-running it? Wouldn't it be even better to just move around and have the cross-section automatically update for us, instead of generating dozens of plots to try to decide which one is actually interesting?

This is where Holoviews's **Streams** come into play. A Stream provides a continuous "stream" of information based on whatever we tell it to watch over. In our case, we want a stream following the cursor (the mouse) whenever it's over our plot, to tell us in which coordinates we are so we can plot the corresponding cross section.

In [ ]:
hv.extension('bokeh')

cmap_temp = "cmo.thermal"
initial_lon = -68.5 # Explained below

# Stream to follow the pointer over the bird's-eye view plot (and then update the lower plot's cross-section)
# Since we are only intersted in the longitude, we'll only track the X position of the mouse
posx_temp = hv.streams.PointerX()

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS

# Widget to select the DEPTH
depth_slider_temp = pn.widgets.DiscreteSlider(name='Depth', 
                                              options=ds_temp.st_ocean.values.tolist(), 
                                              value=ds_temp.st_ocean.values[0], 
                                              width=500, formatter='%.2f (m)')

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS's LINKS

# Watch for changes on the widget's value to update the plot
@pn.depends(depth=depth_slider_temp.param.value)
def update_plot_temp(depth):
    plot = hv.QuadMesh(ds_temp.loc[depth], 
                       kdims=["xt_ocean", "yt_ocean"])
    plot.opts(cmap=cmap_temp,
              title='Temperature at %.2f(m) depth'% (depth), ylabel='Latitude (°N)', xlabel='Longitude (°E)',
              colorbar=True, clabel="°C", 
              width=900, height=250, bgcolor='Silver', 
              framewise=True,
              tools=['hover'], toolbar="above")
    return plot

# To update the cross-plot, we don't need to add the @pn.depends decorator. Why?
# Since we have created a Holoviews Stream, and not a Panel Widget, the @pn decorator is completely useless.
# Instead, we'll let Holoviews automatically call this function whenever the Stream changes values, a few code lines below.
# However, since Holoviews will call this function with the X coordinate of the cursor (provided by PointerX),
# we need the function to be ready to accept the argument.
def update_cross_temp(x):
    # The PointerX stream is extremely precise, so when it calls the function it will not provide a rounded
    # longitude value, say 68.5, but instead it will provide something like 68.4897529683. Thus, we declared
    # above a function to quickly & efficiently find the closest longitude in our data.
    # Further, when the plot loads for the first time, the cursor is still nowhere, and hence x == None.
    # To prevent an error, we established an initial longitude by default for the cross section,
    # in this case, the Drake Passage.
    longitud = closest_lon(x or initial_lon)
    
    plot = hv.QuadMesh(ds_temp.sel(xt_ocean=longitud), # Find the cross-section selected
                       kdims=["yt_ocean", "st_ocean"])
    plot.opts(cmap=cmap_temp,
              title='Temperature cross-section at %.1f°E'% (longitud), ylabel='Depth (m)', xlabel='Latitude (°N)', 
              colorbar=True, clabel="°C", 
              width=900, height=450, bgcolor='Silver', 
              framewise=True, 
              tools=['hover'], toolbar="above",
              invert_yaxis=True) # We must invert the y-axis so the surface (0) is at the top of the plot
    
    return plot
# We can see that update_plot_temp & update_cross_temp have almost identical code. 
# The only difference is on how the arguments are provided, and on the data used to plot, 
# but the idea is basically the same.

# The "issue" we face now is how to know where we are on the above plot when we have moved the cursor out of it.
# Which cross-section is this? Where were we looking at? The easiest solution is to draw a simple dashed-line over the plot,
# indicating which cross-section is being displayed.
# As before, we'll tie the function to the X coordinate provided by the Stream in the code below.
def update_vline_temp(x):
    line = hv.VLine(x or initial_lon) # A Vertical Line
    line.opts(color="MediumSeaGreen", line_dash='dashed')
    return line

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# LONGITUDE LINE
# Here is where we tell Holoviews to call update_vline_temp whenever we move the cursor.
# We tie the DynamicMap to the values provided by the stream posx_temp.

# !!! It is CRUCIAL that the vertical line is THE FIRST one tied to posx_temp. 

# The Stream will be automatically tied to whichever DynamicMap it is first assigned to,
# and it can only track one DynamicMap, so if we assigned it first to the cross-section plot it will be useless. 
# We need it to track the mouse position on the bird's-eye view plot, not over the cross-section, 
# so we must tie it to the vertical line that we will display over the above plot.
vline_temp = hv.DynamicMap(update_vline_temp, streams=[posx_temp])

# TEMPERATURE ANTARCTIC PLOT
plot_temp = hv.DynamicMap(update_plot_temp)

# TEMPERATURE CROSS-SECTION
# Here, we tell Holoviews to call update_cross_temp whenever posx_temp changes.
# Since posx_temp has been tied to the vertical line, update_cross_temp will be called whenever
# the posx_temp & the vertical line change values, just as we wanted
cross_temp = hv.DynamicMap(update_cross_temp, streams=[posx_temp])

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# Build the layout 
# The * in plot_temp * vline_temp is the way of telling Holoviews to plot one thing on top of another.
# In our case, we're telling Holoviews to plot the vertical line above the bird's-eye view plot.
# This is also where, implicitly, we're telling the Stream to track the mouse cursor only on the above plot,
# since that's where the vertical line will be displayed (this, to me, seems almost like magic)
pn.Column(pn.Row(pn.Spacer(width=50), depth_slider_temp), 
          pn.Spacer(height=5),
          plot_temp * vline_temp,
          cross_temp)

As before, we can simply change the data, from:

`ds_temp = dso.temp.sel(yt_ocean=slice(-78,-45)).mean('time')` 

to, say, 

`ds_salt = dso.salt.sel(yt_ocean=slice(-78,-45)).mean('time')` 

and you have yourself the same setup to explore the salinity in  your data!

**Tip**: If you're wondering why not just use beautiful Spanish and call it "datos", as before, instead of ds_temp or ds_salt, it's to prevent naming conflicts within the same Jupyter Notebook. 

If you put several of these interactive plots in the same notebook, and they all share the same names on plots, widgets, etc., they'll start conflicting with one another and stop responding — as you may have noticed already if you've played too much with the notebook. If this happens, just re-run the plot's code and it'll be back on-line. 

To prevent this, I personally just append a \_vble name to every object for the plot: ds**_temp**, vline**_temp**, plot**_temp**. When I want to reuse the code, I copy–paste it, and use Jupyter's `Find & Replace` to replace all \_temp for \_salt within the cell: an effortless way to have all plots working correctly at the same time.

# 3. THE FINAL PLOT-TWIST

Let's do a final touch to our plots, so they're "consistent". We have a vertical longitude line to indicate where is the cross-section we're showing. However, we have no way of tracking how deep we are into the ocean. Why not use our cross-section to show a horizontal line indicating the depth that is being shown on the bird's-eye view?

In [ ]:
hv.extension('bokeh')

cmap_temp = "cmo.thermal"
initial_lon = -68.5

# Stream to follow the pointer over the upper plot (and then update the lower plot's cross-section)
posx_temp = hv.streams.PointerX()

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS

# Widget to select the DEPTH
depth_slider_temp = pn.widgets.DiscreteSlider(name='Depth', options=ds_temp.st_ocean.values.tolist(), 
                                             value=ds_temp.st_ocean.values[0], width=500, formatter='%.2f (m)')


# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS' & STREAMS' LINKS
# Watch for changes on the widgets' values to update the plots

@pn.depends(depth=depth_slider_temp.param.value)
def update_plot_temp(depth):
    plot = hv.QuadMesh(ds_temp.loc[depth], 
                       kdims=["xt_ocean", "yt_ocean"])
    plot.opts(cmap=cmap_temp,
              title='Temperature at %.2f(m) depth'% (depth), ylabel='Latitude (°N)', xlabel='Longitude (°E)',
              colorbar=True, clabel="°C", 
              width=900, height=250, bgcolor='Silver', 
              framewise=True, 
              tools=['hover'], toolbar="above",
              hooks=[set_inactive_tool]) # The hover tool is very uself, but if you don't want it activated by default
                                         # you can just call the set_inactive_tool function defined at the top
                                         # and hook it to this plot to deactivate the hover
    return plot


def update_cross_temp(x):
    longitud = closest_lon(x or initial_lon)
    
    plot = hv.QuadMesh(ds_temp.sel(xt_ocean=longitud), 
                       kdims=["yt_ocean", "st_ocean"])
    plot.opts(cmap=cmap_temp,
              title='Temperature cross-section at %.1f°E'% (longitud), ylabel='Depth (m)', xlabel='Latitude (°N)', 
              colorbar=True, clabel="°C", 
              width=900, height=450, bgcolor='Silver', 
              framewise=True, 
              tools=['hover'], toolbar="above",
              hooks=[set_inactive_tool],
              invert_yaxis=True)
    return plot

def update_vline_temp(x):
    line = hv.VLine(x or initial_lon) # A Vertical Line
    line.opts(color="MediumSeaGreen", line_dash='dashed')
    return line

# The horizontal depth-line will be a mix: as an element, it is exactly like the vertical line; however,
# unlike the vertical line, it depends on a widget, not on the value of a stream. 
# We thus need the @pn.depend decorator to tie the function to the value of the widget,
# so it is automatically called when it changes.
@pn.depends(depth=depth_slider_temp)
def update_hline_temp(depth):
    line = hv.HLine(depth)
    line.opts(color="MediumSeaGreen", line_dash='dashed')
    return line

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# LONGITUDE LINE
# !!! Remember that the vertical line must be the 1st one tied to the Stream
vline_temp = hv.DynamicMap(update_vline_temp, streams=[posx_temp])

# DEPTH LINE
# Again, we don't need to tie the horizontal depth line to a stream, since the function
# update_hline_temp will automatically be called when the widget changes value 
# thanks to the @pn.depends decorator
hline_temp = hv.DynamicMap(update_hline_temp)

# TEMPERATURE ANTARCTIC PLOT
plot_temp = hv.DynamicMap(update_plot_temp)

# TEMPERATURE CROSS-SECTION
cross_temp = hv.DynamicMap(update_cross_temp, streams=[posx_temp])

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# Build the layout
pn.Column(pn.Row(pn.Spacer(width=50), depth_slider_temp), 
          pn.Spacer(height=5),
          plot_temp * vline_temp,
          cross_temp * hline_temp) # As before, the * tells Holoviews to plot hline_temp on top of cross_temp

And now, as we change the depth value in the widget, our depth-line automatically shows us in the cross-section plot at which depth we're exploring in the upper plot ^_^

# 4. THE WORLD DOESN'T TURN AROUND BOKEH

Of course, at this point you may have forgotten that, besides Bokeh, Holoviews can also use the **MatPlotLib** engine. Bokeh is the one providing the zooming, hover, and all that nice, extra functionality, so why chaging to MatPlotLib?

Sadly, there are 2 things that Bokeh sucks at: quivers & stereographic plots. Quivers it can sort of do, but they look just dreadful, so better ignore their existence. Stereographics plots, I haven't figured out how to make them work in Bokeh at all.

Can we have interactivity without Bokeh? Of course! In fact, we don't even need Holoviews to provide interactivity: Panel can also do it. Let's plot some current vectors to take a look.

In [ ]:
# Mean currents in the Southern Ocean, through depth
ds_u = dso.u.sel(yu_ocean=slice(-78,-45)).mean('time')
ds_v = dso.v.sel(yu_ocean=slice(-78,-45)).mean('time')

While we are at it, let's look into some other widgets available (see https://panel.pyviz.org/user_guide/Widgets.html)

In [ ]:
# Bipolar Colormap for the arrows: westward will be blue, eastward red
vector_map = colors.ListedColormap(['RoyalBlue', 'Crimson'])

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS

# Widget to select the DEPTH RANGE
# In this case, the widget lets us select a range of values, from a min to a max,
# not just a single value. This will allow us to average current velocities at different depths,
# which can be extremely useful to understand what's happening.
depth_range_slider_u = pn.widgets.IntRangeSlider(name='Depth Range', 
                                                 start=0,
                                                 # The end value will be determined by the data itself,
                                                 # so we select just the last depth available
                                                 end=np.int(np.rint(ds_u.st_ocean.values[-1])),
                                                 # The initial range when the widget loads
                                                 value=(0, 100), 
                                                 # Incremental step of the widget (in this case 10 m)
                                                 step=10, 
                                                 width=450)

# Widget to select the SCALE for the vectors
# This is another kind of slider, in this case allowing Float values.
# We'll use it to select the "scale" value for matplotlib's quiver function,
# so we can make the arrows smaller or larger as needed.
scale_slider_u = pn.widgets.FloatSlider(name='Vector Scale', 
                                        start=0.5, end=10, step=0.5, value=5, 
                                        width=150)

# Widget to select the STEP to sample the vectors.
# We can have too many arrows on screen, leading to overplotting and not seeing anything,
# or too few and missing information. We can provide an Integer slider to select 
# how many arrows we want displayed.
# A value of 3 will show only every third arrow on the plot, a 5 every fifth, etc.
step_slider_u = pn.widgets.IntSlider(name='Sampling Step', 
                                     start=1, end=6, step=1, value=5, 
                                     width=150)

# ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
# WIDGETS's LINKS

# As always, we use the @pn.depend decorator to tell the function which widgets to watch.
# Whenever ANY of them changes its value, the function will be called to regenerate the plot.
@pn.depends(sel_depths=depth_range_slider_u, 
            sel_scale=scale_slider_u,
            sel_step=step_slider_u)  
def plot_currents(sel_depths, sel_scale, sel_step):
    # The rest of the function's code is just plain matplotlib-code. Nothing to do with panel, holoviews, or any of it.
    plt.figure(figsize=(13.5,4))
    ax = plt.subplot(1, 1, 1, projection=ccrs.Mercator(central_longitude=-80))
    
    ax.add_feature(cartopy.feature.OCEAN, zorder=0, color='Azure')
    ax.add_feature(cartopy.feature.LAND, zorder=0, color='Khaki')

    # Compute mean velocities for the arrows.
    # It must be done within the function because we need to:
    #  — slice the depths according to the range selected on the widget
    #  — sub-select the resulting data according to how many arrows we want displayed
    u_vels = ds_u.loc[sel_depths[0]:sel_depths[1]].\
                              mean('st_ocean')[::sel_step, ::sel_step]
    v_vels = ds_v.loc[sel_depths[0]:sel_depths[1]].\
                              mean('st_ocean')[::sel_step, ::sel_step]
    
    # Base the color of the arrows on the direction of the u-velocity (bipolar)
    flow_directions = np.sign(u_vels)
    
    # Plot the quivers, passing only every sel_step X & Y coordinate,
    # and the scale selected on the widget.
    # Again, this is just matplotlib's quiver code: panel has only intervened to provide values
    # during the function call
    ax.quiver(ds_u.xu_ocean[::sel_step].values, ds_u.yu_ocean[::sel_step].values,
              u_vels.values, v_vels.values, flow_directions, scale=sel_scale, 
              cmap=vector_map, transform=ccrs.PlateCarree())
    
    ax.set_title(f"Currents from {sel_depths[0]:d} to {sel_depths[1]:d} (m) depth", 
                 fontsize=20, pad=20)
    
    # ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ^_^ ~~~~~ ~~~~~ ~~~~~ ~~~~~ ~~~~~
    curr_fig=plt.gcf()  
    # Since curr_fig.tight_layout() doesn't work for some unholy reason, 
    # we're forced to manually destroy the margins around our plot before returning it
    curr_fig.subplots_adjust(left=0, right=1, top=0.95, bottom=0)
    # We need to close the current figure before returning it, to prevent matplotlib from
    # displaying it twice
    plt.close(curr_fig) 
    return curr_fig

# And THAT'S ALL we need to do to provide interactivity: create some widgets & tie them to a function.
# Since panel will automatically call the function to regenerate the plot whenever the 
# widgets change value, all is left to do is build a nice layout for all our widgets
pn.Column(pn.Row(depth_range_slider_u, scale_slider_u, step_slider_u),
          plot_currents) # We can simply pass the function to panel, and panel will take control and 
                         # call it when necessary

We can play around as much as we want to investigate our currents, at whichever depth & resolution we want. We've used the same concepts we learned with Bokeh, but now we're using the capabilities of Matplotlib and the interactivity provided by Panel.

# 5. WHAT ABOUT THOSE NICE TIME-SERIES PLOTS?

Now that we know how to plot things one on top of another, you might have remembered the time-series plot we did on the basic level and wondered if you could use this technique to take a look at the time-series of 2 (or more!) experiments at the same time. The answer is: of course. Let's build a function to do just that automatically for us.

In [ ]:
def compara(ctrl_data, exp_data, val_dim):
    """
    Creates an interactive plot for comparison, overlaying the data of the control run & the experiment 
    on the same plot
        
    Parameters:
    -----------
        ctrl_data: dataset with the control data
        exp_data: dataset with the model run data 
        val_dim: string with the name of the data_var(iable) to plot
    """
        
    hv.extension('bokeh')

    # Control run
    hv_ds_c = hv.Dataset(ctrl_data)
    wi_c = hv_ds_c.to(hv.Curve, kdims=['time'], vdims=[val_dim], label="Control", dynamic=True)
    wi_c.opts(framewise=True)
              
    # Experiment run
    hv_ds = hv.Dataset(exp_data)
    wi = hv_ds.to(hv.Curve, kdims=['time'], vdims=[val_dim], label="Experiment", dynamic=True)
    wi.opts(framewise=True)
    
    # Build the layout
    layout = wi * wi_c
    # Remove the common title of both plots, since the slider (or widget) already is displaying the same information
    layout.opts(title='', width=600, height=600, padding=(0, 0.05), 
                fontsize={'labels':12, 'yticks':12, 'title':12})
    # Centre the widget on top of both plots
    hv_panel = pn.panel(layout, center=True, widget_location='top')
    # Make the widget wider, for more precise control
    hv_panel[1][0][1][0].width = 500
    # Show the result
    return hv_panel

Easy! Nothing new on all the code we've written: we've just put a few ideas together. Now, let's take a look at what happens:

In [ ]:
# Control run data
mean_temps_c = (dso_c.temp.mean(['yt_ocean', 'xt_ocean'])) - 273.15
# Experiment run data
mean_temps = (dso.temp.mean(['yt_ocean', 'xt_ocean'])) - 273.15

compara(ctrl_data=mean_temps_c, exp_data=mean_temps, val_dim="temp")

# 6. CONCLUSION

The possibilities for data exploration are now endless thanks to the wide range of widgets available; our ingenuity the only limit! We'll look into some "wilder" ideas on the final tutorial: the Advanced level.